# Implementing and vectorizing a Maximum Likelihood model 

This notebook walks through the process of coding, testing, estimating, and vectorizing a Maximum Likelihood (MLE) model "from scratch" using scipy's numerical optimization package. Not all MLE models are available in pre-cooked packages, so this skill is necessary for some research topics.

We will only be touching a simple model here (regular probit), so as not to get bogged down in extraneous complexities. This general method of estimation, testing, and vectorization will however work for a wide range of MLE models.

**Probit**

The basic probit model takes a binary dependent variable, $Y$, and assumes that $Pr(Y=1 | X) = \Phi(X^T \beta)$ where $X$ is the matrix of independent variables, $\beta$ is the vector of parameters to estimate, and $\Phi$ is the CDF of the standard normal distribution. We want to take the likelihood function $L=PR(\beta|X, Y)$, and maximize it over $\beta$ to get the most likely $\beta$ paramaters given the data $X,Y$. We usually use the log of the likelihood function in practice, because it is simpler in both math and computation, and the maximum point is the same. The probit log likelihood is as follows:

$$ln L(\beta|X,Y) = \sum_{i=1}^n[y_i ln \Phi(x_i'\beta)+(1-y_i)ln(1-\Phi(x_i'\beta)) ]$$

Which we can translate into a naive python function like this:

In [10]:
import numpy as np
from scipy.stats import norm

def LogLikeProbit(betas, y, x):
    """
    Probit Log Likelihood function
    Very slow naive Python version
    Input:
        betas is a np.array of parameters
        y is a one dimensional np.array of endogenous data
        x is a 2 dimensional np.array of exogenous data
            First vertical colmn of X is assumed to be constant term,
            corresponding to betas[0]
    returns:
        negative of log likehood value (scalar)
    """
    result = 0
    #Sum operation
    for i in range(0, len(y)):
        #Get X_i * Beta value
        xb = np.dot(x[i], betas)
        
        #compute both binary probabilities from xb     
        #Add to total log likelihood
        llf = y[i]*np.log(norm.cdf(xb)) + (1-y[i])*np.log(1 - norm.cdf(xb))
        result += llf
    return -result


Note that we return the negative value of the result because we want to maximize over this function, and numerical optimizers are traditionally minimizers. Minimizing over the negative values will be the same as maximizing the function.

**Generating a testing environment for your model**

When creating a model from scratch, we need to know it is correct on data where we know the real values and distributions. Here is artificial data to test our probit model on:

In [11]:
######################
#ARTIFICIAL DATA
######################

#sample size
n = 1000

#random generators
z1 = np.random.randn(n)
z2 = np.random.randn(n)

#create artificial exogenous variables 
x1 = 0.8*z1 + 0.2*z2
x2 = 0.2*z1 + 0.8*z2
#create error term
u = 2*np.random.randn(n)

#create endogenous variable from x1, x2 and u
ystar = 0.5 + 0.75*x1 - 0.75*x2 + u

#create latent binary variable from ystar
def create_dummy(data, cutoff):
    result = np.zeros(len(data))
    for i in range(0, len(data)):
        if data[i] >= cutoff:
            result[i] = 1
        else:
            result[i] = 0
    return result

#get latent LHS variable
y = create_dummy(ystar, 0.5)

#prepend vector of ones to RHS variables matrix
#for constant term
const = np.ones(n)
x = np.column_stack((const, np.column_stack((x1, x2))))



**Testing the model**

We can now maximize the probit log likelihood to get the most likely vector of parameters given the artificial data using scipy's powerful numerical optimization library:

In [12]:
from scipy.optimize import minimize

#create beta hat vector to maximize on
#will store the values of maximum likelihood beta parameters
#Arbitrarily initialized to all zeros
bhat = np.zeros(len(x[0]))

#unvectorized MLE estimation
probit_est = minimize(LogLikeProbit, bhat, args=(y,x), method='nelder-mead')

#print vector of maximized betahats
probit_est['x']

array([-0.01675211,  0.41512549, -0.30328131])

In [13]:
import statsmodels.tools.numdiff as smt
import scipy as sc

#Get inverse hessian for Cramer Rao lower bound
b_estimates = probit_est['x']
Hessian = smt.approx_hess3(b_estimates, LogLikeProbit, args=(y,x))
invHessian = np.linalg.inv(Hessian)

#Standard Errors from C-R LB
#from diagonal elements of invHessian
SE = np.zeros(len(invHessian))
for i in range(0, len(invHessian)):
    SE[i] =  np.sqrt(invHessian[i,i])
    
#t and p values
t_statistics = (b_estimates/SE)
pval = (sc.stats.t.sf(np.abs(t_statistics), 999)*2)

print("Beta Hats: ", b_estimates)
print("SE: ", SE)
print("t stat: ", t_statistics)
print("P value: ", pval)

Beta Hats:  [-0.01675211  0.41512549 -0.30328131]
SE:  [0.0403861  0.05649399 0.05742784]
t stat:  [-0.41479881  7.34813595 -5.28108488]
P value:  [6.78378253e-01 4.17250373e-13 1.57589878e-07]


In [16]:
result = 0
for i in range(0, len(y)):
        xb = np.dot(x[i], bhat)
        llf = y[i]*np.log(norm.cdf(xb)) + (1-y[i])*np.log(1 - norm.cdf(xb))
        result += llf

In [17]:
xb = np.dot(x, bhat)
result = np.sum(    
        (y==1)*np.log(1 - norm.cdf(xb)) + 
        (y==0)*np.log(norm.cdf(xb))
        )

In [18]:
xb = np.dot(x, bhat)
result = np.sum(np.log(  
        (y==1)*(1 - norm.cdf(xb)) + 
        (y==0)*(norm.cdf(xb))
        ))

In [19]:
def VectorizedProbitLL(betas, y, x):
    xb = np.dot(x, betas)
    result = np.sum(np.log(  
        (y==0)*(1 - norm.cdf(xb)) + 
        (y==1)*(norm.cdf(xb))
        ))
    return -result

In [20]:
import timeit

%timeit minimize(VectorizedProbitLL, bhat, args=(y,x), method='nelder-mead')

60.7 ms ± 2.83 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [21]:
%timeit minimize(LogLikeProbit, bhat, args=(y,x), method='nelder-mead')

44.8 s ± 11.4 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%timeit minimize(VectorizedProbitLL, bhat, args=(y,x), method='bfgs')

The slowest run took 16.23 times longer than the fastest. This could mean that an intermediate result is being cached.
66.1 ms ± 83 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


BFGS is about twice as fast as Nelder-Mead in this case.

Note that the BFGS algorithm sometimes throws a warning about a division by 0 here. This most likely comes from the fact that all our data is binary; methods that have to estimate derivatives have a harder time with sparse or binary data (among others).